#  Домашнее задание к занятию "Тематическое моделирование"

### Классификация по тональности
В этом домашнем задании вам предстоит классифицировать по тональности отзывы на банки с сайта banki.ru.

Данные содержат непосредственно тексты отзывов, некоторую дополнительную информацию, а также оценку по шкале от 1 до 5.

Тексты хранятся в json-ах в массиве responses.

Посмотрим на примере отзыва: возьмите для удобства ноутбук, размещенный в папке репозитория.

### Часть 1. Анализ текстов
1. Посчитайте количество отзывов в разных городах и на разные банки
1. Постройте гистограмы длин слов в символах и в словах
1. Найдите 10 самых частых:
- слов
- слов без стоп-слов
- лемм
- существительных
4. Постройте кривые Ципфа и Хипса
5. Ответьте на следующие вопросы:
- какое слово встречается чаще, "сотрудник" или "клиент"?
- сколько раз встречается слова "мошенничество" и "доверие"?
6. В поле "rating_grade" записана оценка отзыва по шкале от 1 до 5. Используйте меру $tf-idf$, для того, чтобы найти ключевые слова и биграмы для положительных отзывов (с оценкой 5) и отрицательных отзывов (с оценкой 1)

### Часть 2. Тематическое моделирование
1. Постройте несколько тематических моделей коллекции документов с разным числом тем. Приведите примеры понятных (интерпретируемых) тем.
2. Найдите темы, в которых упомянуты конкретные банки (Сбербанк, ВТБ, другой банк). Можете ли вы их прокомментировать / объяснить? Эта часть задания может быть сделана с использованием gensim.

### Часть 3. Классификация текстов
- Сформулируем для простоты задачу бинарной классификации: будем классифицировать на два класса, то есть, различать резко отрицательные отзывы (с оценкой 1) и положительные отзывы (с оценкой 5).

1. Составьте обучающее и тестовое множество: выберите из всего набора данных N1 отзывов с оценкой 1 и N2 отзывов с оценкой 5 (значение N1 и N2 – на ваше усмотрение). Используйте sklearn.model_selection.train_test_split для разделения множества отобранных документов на обучающее и тестовое.
1. Используйте любой известный вам алгоритм классификации текстов для решения задачи и получите baseline. Сравните разные варианты векторизации текста: использование только униграм, пар или троек слов или с использованием символьных $n$-грам.
1. Сравните, как изменяется качество решения задачи при использовании скрытых тем в качестве признаков:
- 1-ый вариант: $tf-idf$ преобразование (sklearn.feature_extraction.text.TfidfTransformer) и сингулярное разложение (оно же – латентый семантический анализ) (sklearn.decomposition.TruncatedSVD),
- 2-ой вариант: тематические модели LDA (sklearn.decomposition.LatentDirichletAllocation). Используйте accuracy и F-measure для оценки качества классификации.

В ноутбуке, размещенном в папке репозитория. написан примерный Pipeline для классификации текстов.

Эта часть задания может быть сделана с использованием sklearn.

## Импорт библиотек

In [1]:
import json

import bz2
import regex 
from tqdm import tqdm
from scipy import sparse

import pprint

In [2]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Данные

In [3]:
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [02:26, 1368.77it/s]



Посмотрим на пример отзыва:

In [4]:
responses[99]

{'city': 'г. Саратов',
 'rating_not_checked': False,
 'title': 'Карта ко вкладу',
 'num_comments': 0,
 'bank_license': 'лицензия № 880',
 'author': 'ronnichka',
 'bank_name': 'Югра',
 'datetime': '2015-06-03 20:56:57',
 'text': 'Здравствуйте! Хотела написать, что мне месяц не выдают карту ко вкладу, ссылаясь на "нам же их из Самары везут" (на секундочку 5 часов езды от нашего города). Но! Прочитала, что людям 3,5 месяцев не выдают карту, и поняла, что у меня все хорошо, пока что. И подарок мне дали, и кулер в отделении есть. Так что я, конечно, готова ждать. Правда хотелось бы не очень долго.',
 'rating_grade': 3}

### Решение

### Часть 1. Анализ текстов
1. Посчитайте количество отзывов в разных городах и на разные банки
2. Постройте гистограмы длин слов в символах и в словах
3. Найдите 10 самых частых:
- слов
- слов без стоп-слов
- лемм
- существительных

4. Постройте кривые Ципфа и Хипса
5. Ответьте на следующие вопросы:
- какое слово встречается чаще, "сотрудник" или "клиент"?
- сколько раз встречается слова "мошенничество" и "доверие"?

6. В поле "rating_grade" записана оценка отзыва по шкале от 1 до 5. Используйте меру $tf-idf$, для того, чтобы найти ключевые слова и биграмы для положительных отзывов (с оценкой 5) и отрицательных отзывов (с оценкой 1)


Поместим данные в датафрейм при помощи pandas, а далее можно уже будет работать с таблицей

In [5]:
data_responses = pd.DataFrame(responses)
data_responses

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
0,г. Москва,False,Жалоба,0,лицензия № 2562,uhnov1,Бинбанк,2015-06-08 12:50:54,Добрый день! Я не являюсь клиентом банка и пор...,NaN
1,г. Новосибирск,False,Не могу пользоваться услугой Сбербанк он-лайн,0,лицензия № 1481,Foryou,Сбербанк России,2015-06-08 11:09:57,Доброго дня! Являюсь держателем зарплатной кар...,NaN
2,г. Москва,False,Двойное списание за один товар.,1,лицензия № 2562,Vladimir84,Бинбанк,2015-06-05 20:14:28,Здравствуйте! Дублирую свое заявление от 03.0...,NaN
3,г. Ставрополь,False,Меняют проценты комиссии не предупредив и не ...,2,лицензия № 1481,643609,Сбербанк России,2015-06-05 13:51:01,Добрый день!! Я открыл расчетный счет в СберБа...,NaN
4,г. Челябинск,False,Верните денежные средства за страховку,1,лицензия № 2766,anfisa-2003,ОТП Банк,2015-06-05 10:58:12,"04.03.2015 г. взяла кредит в вашем банке, заяв...",NaN
...,...,...,...,...,...,...,...,...,...,...
153494,None,False,Не все так страшно,0,лицензия № 2557,Вера,Ситибанк,2005-05-18 14:38:00,"Слишком большой банк, не всегда учитывает нашу...",5.0
153495,None,False,А вы договор читали?,0,лицензия № 2557,Kirill,Ситибанк,2005-04-14 15:25:00,"Вы сами виноваты в своих проблемах, кроме пред...",3.0
153496,None,False,"Филиал банка ""ОВК"" в г. Иваново",1,лицензия № 2272,AlexU-post,Росбанк,2005-04-08 14:07:00,Уважаемое руководство банка «ОВК»! Я проживаю ...,1.0
153497,None,False,В Альфа-Банк больше обращаться не буду.,0,лицензия № 1326,Ирина,Альфа-Банк,2005-04-14 15:16:00,Пробовала 10.04.05 оформить товар в кредит в А...,1.0


In [6]:
data_responses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153499 entries, 0 to 153498
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   city                138325 non-null  object 
 1   rating_not_checked  153499 non-null  bool   
 2   title               153499 non-null  object 
 3   num_comments        153499 non-null  int64  
 4   bank_license        153498 non-null  object 
 5   author              153479 non-null  object 
 6   bank_name           153499 non-null  object 
 7   datetime            153499 non-null  object 
 8   text                153499 non-null  object 
 9   rating_grade        88658 non-null   float64
dtypes: bool(1), float64(1), int64(1), object(7)
memory usage: 10.7+ MB


In [7]:
quantity_banks = data_responses.groupby(by = 'city').count().sort_values(by='text', ascending=False )['text']
quantity_banks

city
г. Москва                           55354
г. Санкт-Петербург                  14342
г. Екатеринбург                      2337
г. Новосибирск                       2300
г. Нижний Новгород                   1976
                                    ...  
г. Москва, Ногинск (Москва)             1
г. Москва, Новосибирск                  1
г. Москва, Николоямская ул., 31         1
г. Москва, Московская область           1
г. … или другой населенный пункт        1
Name: text, Length: 5823, dtype: int64

In [8]:
quantity_banks = data_responses.groupby(by = 'bank_name').count().sort_values(by='text', ascending=False )['text']
quantity_banks

bank_name
Сбербанк России           26327
Альфа-Банк                10224
ВТБ 24                     8185
Русский Стандарт           7943
Хоум Кредит Банк           7549
                          ...  
Северный Народный Банк        1
Сельмашбанк                   1
Сиббизнесбанк                 1
Сибконтакт                    1
Мигом                         1
Name: text, Length: 670, dtype: int64

#### 1 Посчитайте количество отзывов в разных городах и на разные банки

In [9]:
data_responses_2 = data_responses

In [10]:
# количество нулевых = неизвестных значений городов
data_responses_2[data_responses_2.city.isnull()].head()

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
89526,None,False,"Халатность сотрудника, обман потребителя",3,лицензия № 1481,марс13,Сбербанк России,2013-03-06 10:26:00,В январе мной было подано заявление на досрочн...,1.0
131055,None,False,"Любимый мой банк, но в последнее время удивляет",7,лицензия № 3279,Lzt06,Национальный Банк «Траст»,2010-09-13 21:34:00,"С удовольствием обслуживаюсь с 2009 г. в НБ ""Т...",3.0
138284,None,False,Из-за одного некомпетентного сотрудника портит...,5,лицензия № 1460,1й2ц3у4,Восточный Экспресс Банк,2009-07-03 01:06:00,Как один человек может испортить репутацию бан...,NaN
138285,None,False,Хамство кассира в отделении на Автозаводской,7,лицензия № 2771,sergor,Юниаструм Банк,2009-07-01 18:39:00,"Недавно был в отделении на Автозаводской, Моск...",1.0
138286,None,False,МЕП,61,лицензия № 2557,Квинки,Ситибанк,2009-06-29 16:42:00,"Это, собственно, не отзыв, это - размышлизм......",NaN


In [11]:
# замена строк на Неизвестный город и без рейтинга на ноль
for row in range(0, len(data_responses_2)):
    if data_responses_2.city[row] == None: 
        data_responses_2.city[row] = 'Неизвестный'
    elif data_responses_2['rating_grade'].isnull()[row]: 
        data_responses_2['rating_grade'][row] = 0
    else:
        continue

<ipython-input-11-c37f173ccc42>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_responses_2['rating_grade'][row] = 0
<ipython-input-11-c37f173ccc42>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_responses_2.city[row] = 'Неизвестный'


In [12]:
data_responses_2.head()

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
0,г. Москва,False,Жалоба,0,лицензия № 2562,uhnov1,Бинбанк,2015-06-08 12:50:54,Добрый день! Я не являюсь клиентом банка и пор...,0.0
1,г. Новосибирск,False,Не могу пользоваться услугой Сбербанк он-лайн,0,лицензия № 1481,Foryou,Сбербанк России,2015-06-08 11:09:57,Доброго дня! Являюсь держателем зарплатной кар...,0.0
2,г. Москва,False,Двойное списание за один товар.,1,лицензия № 2562,Vladimir84,Бинбанк,2015-06-05 20:14:28,Здравствуйте! Дублирую свое заявление от 03.0...,0.0
3,г. Ставрополь,False,Меняют проценты комиссии не предупредив и не ...,2,лицензия № 1481,643609,Сбербанк России,2015-06-05 13:51:01,Добрый день!! Я открыл расчетный счет в СберБа...,0.0
4,г. Челябинск,False,Верните денежные средства за страховку,1,лицензия № 2766,anfisa-2003,ОТП Банк,2015-06-05 10:58:12,"04.03.2015 г. взяла кредит в вашем банке, заяв...",0.0


In [13]:
from nltk.tokenize import word_tokenize

In [14]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
from collections import Counter
corpus = [token for respon in data_responses_2.city for token in word_tokenize(respon) if token not in punctuation]
print(len(corpus))
corpus[:10]

303659


['г.',
 'Москва',
 'г.',
 'Новосибирск',
 'г.',
 'Москва',
 'г.',
 'Ставрополь',
 'г.',
 'Челябинск']

In [17]:
freq_dict = Counter(corpus)

freq_dict_sorted= sorted(freq_dict.items(), key=lambda x: -x[1])
# freq_dict.most_common(20)

In [18]:
freq_dict_sorted

[('г.', 138574),
 ('Москва', 56268),
 ('Неизвестный', 15174),
 ('Санкт-Петербург', 14525),
 ('Екатеринбург', 2365),
 ('Новосибирск', 2338),
 ('Новгород', 2273),
 ('Нижний', 2261),
 ('Ростов-на-Дону', 1926),
 ('Самара', 1778),
 ('Челябинск', 1541),
 ('Казань', 1470),
 ('Краснодар', 1316),
 ('Уфа', 1279),
 ('Красноярск', 1246),
 ('Воронеж', 1235),
 ('москва', 1232),
 ('Пермь', 1182),
 ('Волгоград', 1036),
 ('Омск', 1011),
 ('Саратов', 961),
 ('Калининград', 764),
 ('Иркутск', 763),
 ('обл', 753),
 ('Ставрополь', 709),
 ('область', 702),
 ('Хабаровск', 701),
 ('Тюмень', 670),
 ('Владивосток', 651),
 ('Барнаул', 623),
 ('Ярославль', 617),
 ('Тула', 602),
 ('Томск', 580),
 ('Рязань', 558),
 ('Ульяновск', 516),
 ('Тольятти', 515),
 ('Иваново', 472),
 ('Липецк', 469),
 ('Ижевск', 463),
 ('Тверь', 457),
 ('Мурманск', 442),
 ('Оренбург', 438),
 ('Пенза', 438),
 ('Московская', 414),
 ('Сочи', 407),
 ('Кемерово', 402),
 ('Магнитогорск', 400),
 ('Белгород', 398),
 ('Чебоксары', 395),
 ('Владимир',

In [19]:
freq_dict_sorted[2]

('Неизвестный', 15174)

#### Второй более простой способ расчёта отзывов по городам и по банкам

In [20]:
data_responses.city.value_counts()

г. Москва                                         55354
Неизвестный                                       15174
г. Санкт-Петербург                                14342
г. Екатеринбург                                    2337
г. Новосибирск                                     2300
                                                  ...  
г. ОРЁЛ                                               1
г. Апатиты, (Мурманская область)                      1
г. г. Заполярный Мурманская область (Мурманск)        1
г. -                                                  1
г. Москва (думаю там находится их справочная)         1
Name: city, Length: 5824, dtype: int64

In [21]:
data_responses.bank_name.value_counts()

Сбербанк России                     26327
Альфа-Банк                          10224
ВТБ 24                               8185
Русский Стандарт                     7943
Хоум Кредит Банк                     7549
                                    ...  
Кошелев-Банк                            1
Новокузнецкий Муниципальный Банк        1
Современные Стандарты Бизнеса           1
Красбанк                                1
Донской Народный Банк                   1
Name: bank_name, Length: 670, dtype: int64

#### 2 Постройте гистограмы длин слов в символах и в словах

In [18]:
data_responses_3 = data_responses_2

In [23]:
data_responses_3.head()

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
0,г. Москва,False,Жалоба,0,лицензия № 2562,uhnov1,Бинбанк,2015-06-08 12:50:54,Добрый день! Я не являюсь клиентом банка и пор...,0.0
1,г. Новосибирск,False,Не могу пользоваться услугой Сбербанк он-лайн,0,лицензия № 1481,Foryou,Сбербанк России,2015-06-08 11:09:57,Доброго дня! Являюсь держателем зарплатной кар...,0.0
2,г. Москва,False,Двойное списание за один товар.,1,лицензия № 2562,Vladimir84,Бинбанк,2015-06-05 20:14:28,Здравствуйте! Дублирую свое заявление от 03.0...,0.0
3,г. Ставрополь,False,Меняют проценты комиссии не предупредив и не ...,2,лицензия № 1481,643609,Сбербанк России,2015-06-05 13:51:01,Добрый день!! Я открыл расчетный счет в СберБа...,0.0
4,г. Челябинск,False,Верните денежные средства за страховку,1,лицензия № 2766,anfisa-2003,ОТП Банк,2015-06-05 10:58:12,"04.03.2015 г. взяла кредит в вашем банке, заяв...",0.0


In [16]:
from nltk.tokenize import word_tokenize

In [25]:
data_responses_3['len_chars'] = 0
data_responses_3['len_words'] = 0
for i, each_response in enumerate(data_responses_3.text):
  data_responses_3['len_chars'][i] = len(each_response)
  data_responses_3['len_words'][i] = len(word_tokenize(each_response))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [26]:
data_responses_3[['city', 'text', 'len_chars', 'len_words']].sort_values(by='len_words')

,city,text,len_chars,len_words
153116,Неизвестный,http://www.dp.ru/publication/2005-06-24/81877,45,3
151714,Неизвестный,Банк больше не работает с обменниками..,39,6
153286,Неизвестный,Охранники у банка слишком злые.,31,6
138228,г. Челябинск,"Вот, собственно, и всё.",23,7
152261,Неизвестный,Быстро получил 150.000 без каких-либо проблем.,46,7
...,...,...,...,...
26023,г. Москва,Как и обещал в своем предыдущем отзыве от 29 о...,27364,4561
86611,г. Москва,История за 10 лет... Напишу кратко (это действ...,24604,4786
38104,г. Москва,Введение оно же вывод. Данный отзыв содержит к...,27220,5042
118257,г. Москва,Весь пост (крайне объёмный) опубликован мной в...,29679,5243


#### 3 Найдите 10 самых частых:
- слов
- слов без стоп-слов
- лемм
- существительных

Ради интереса выявим самые частоиспользуемые символы

In [19]:
chars = [each_char for each_response in data_responses_3.text for each_char in each_response.strip().lower()]

In [20]:
chars[:5]

['д', 'о', 'б', 'р', 'ы']

In [21]:
freq_chars = nltk.FreqDist(chars)

In [22]:
freq_chars.most_common()[:10]

[(' ', 36599529),
 ('о', 20228699),
 ('е', 16339230),
 ('а', 15999296),
 ('н', 13505239),
 ('и', 12827830),
 ('т', 12748087),
 ('с', 9453519),
 ('р', 8753062),
 ('л', 7922451)]

Самые частовстречающиеся слова топ-10

In [23]:
words = []
for each_response in data_responses_3.text:
  words.append(each_response.lower())
words_str = ''.join(words)

In [24]:
words_token = word_tokenize(words_str) 

In [25]:
words_token[:5]

['добрый', 'день', '!', 'я', 'не']

In [26]:
freq_word = nltk.FreqDist(words_token)

In [27]:
freq_word.most_common()[:10]

[(',', 3348588),
 ('.', 2026121),
 ('в', 1315951),
 ('и', 1054301),
 ('не', 850619),
 ('на', 703279),
 ('что', 621603),
 ('я', 569537),
 ('с', 453473),
 ('по', 364744)]

Самые частовстречающиеся слов без стоп-слов топ-10

In [28]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
from nltk.corpus import stopwords
print(stopwords.words('russian'))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [30]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [31]:
noise = stopwords.words('russian') + list(punctuation)

In [32]:
words_token_no_stopwords = words_token
for i, elem in enumerate(words_token_no_stopwords):
  if elem in noise:
    words_token_no_stopwords.pop(i)

KeyboardInterrupt: 

Уберём знаки пунктуации, которые являются повторением тех, которые есть в punctuation, но не удалились

In [33]:
while "..." in words_token_no_stopwords: 
  words_token_no_stopwords.remove("...")
while  "''" in words_token_no_stopwords: 
  words_token_no_stopwords.remove("''")
while "``" in words_token_no_stopwords: 
  words_token_no_stopwords.remove("``")

KeyboardInterrupt: 

In [37]:
words_token_no_stopwords[:10]

['добрый',
 'день',
 'я',
 'являюсь',
 'клиентом',
 'банка',
 'поручителем',
 'кредитному',
 'договору',
 'а']

In [38]:
freq_word_nostop = nltk.FreqDist(words_token_no_stopwords)

In [39]:
freq_word_nostop.most_common()[:10]

[(',', 3342082),
 ('.', 2022112),
 ('в', 1313676),
 ('и', 1052494),
 ('не', 849223),
 ('на', 702001),
 ('что', 621241),
 ('я', 568798),
 ('с', 452532),
 ('по', 363966)]

Самые частовстречающиеся лемм топ-10


In [34]:
!pip install pymorphy2

In [35]:
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()

In [44]:
len(words_token_no_stopwords)

43758199

In [36]:
words_token_lemma = words_token_no_stopwords

In [40]:
for i, word in enumerate(words_token_lemma):
  words_token_lemma[i] = pymorphy2_analyzer.parse(word)[0].normal_form

KeyboardInterrupt: 

In [45]:
# проверка работы pymorphy
# pymorphy2_analyzer.parse('лошади')[0].normal_form

In [42]:
words_token_lemma[:10]

['добрый',
 'день',
 'я',
 'являться',
 'клиент',
 'банк',
 'поручитель',
 'кредитный',
 'договор',
 'а']

In [43]:
len(words_token_lemma)

43758199

In [46]:
freq_word_lemma = nltk.FreqDist(words_token_lemma)

In [47]:
freq_word_lemma.most_common()[:10]

[(',', 3342082),
 ('.', 2022112),
 ('в', 1314297),
 ('и', 1052494),
 ('не', 849223),
 ('на', 702001),
 ('что', 622144),
 ('я', 589219),
 ('с', 454791),
 ('по', 363966)]

Самые частовстречающиеся существительных топ-10

In [55]:
# принцип алгоритма по отбору существительных
# p = pymorphy2_analyzer.parse('идти')
# p

[Parse(word='идти', tag=OpencorporaTag('INFN,impf,intr'), normal_form='идти', score=1.0, methods_stack=((DictionaryAnalyzer(), 'идти', 1696, 0),))]

In [68]:
# p[0].tag

OpencorporaTag('INFN,impf,intr')

In [71]:
# p[0].score

1.0

In [70]:

# 'NOUN' in p[0].tag

False

In [125]:
# проходим по словам-леммам, которые мы подготовили на прошлом шаге
#  далее пропускаем их через pymorphy2_analyzer, и проверяем часть речи, если tag NOUN и при этом оценка более 50 %, 
# то можно причислять это слово к существительному и записывать в список, а потом расчитаем частоту данных слов при помощи FreqDist

i = 0
nouns = []

for word in words_token_lemma:
    p = pymorphy2_analyzer.parse(word)[0]
    if ('NOUN' in p.tag) and (p.score > 0.5):
        nouns[i] = word
        i += 1

IndexError: list assignment index out of range

In [114]:
freq_nouns = nltk.FreqDist(nouns.values())

In [115]:
freq_nouns.most_common()[:10]

[]

#### 4 Постройте кривые Ципфа и Хипса

##### Закон Ципфа
Эмпирическая закономерность: если все слова корпуса текста упорядочить по убыванию частоты их использования, то частота n-го слова в таком списке окажется приблизительно обратно пропорциональной его порядковому номеру n. Иными словами, частотность слов убывает очень быстро.


В любом достаточно большом тексте ранг слова обратно пропорционален его частоте: $f = \frac{a}{r}$

$f$ – частота слова, $r$  – ранг слова, $a$  – параметр, для славянских языков – около 0.07

In [113]:
freqs = list(freq_nouns.values())
freqs = sorted(freqs, reverse = True)

fig, ax = plt.subplots()
ax.plot(freqs[:300], range(300))
plt.show()

###### Закон Хипса

С увеличением длины текста (количества токенов), количество слов увеличивается в соответствии с законом: $|V| = K*N^b$


$N$  –  число токенов, $|V|$  – количество слов в словаре, $K, b$  –  параметры, обычно $K \in [10,100], b \in [0.4, 0.6]$


Закон Хипса -- обратная сторона закона Ципфа. Он описывает, что чем больше корпус, тем меньше новых слов добавляется с добавлением новых текстов. В какой-то момент корпус насыщается.

In [116]:
from tqdm import tqdm

cnt = Counter()
n_words = []
n_tokens = []
tokens = []

for index, row in tqdm(data_responses_3.iterrows(), total = len(data_responses_3)):
    tokens = word_tokenize(row['text'])
    cnt.update([token for token in tokens if token not in punctuation])
    n_words.append(len(cnt))
    n_tokens.append(sum(cnt.values()))

100%|█████████████████████████████████████████████████████████████████████████| 153499/153499 [14:46<00:00, 173.13it/s]


In [118]:
fig, ax = plt.subplots()
ax.plot(n_tokens, n_words)
plt.show()

#### 5 Ответьте на следующие вопросы:
- какое слово встречается чаще, "сотрудник" или "клиент"?
- сколько раз встречается слова "мошенничество" и "доверие"?

In [ ]:
word_1 = "сотрудник"
word_2 = "клиент"
for i in freq_nouns:
    if word_1 in i:
        quantity_1 = i[1]
    elif word_2 in i:
        quantity_2 = i[1]
        
if quantity_1 > quantity_2:
    print('сотрудник встречается чаще')
elif quantity_1 < quantity_2:
        print('клиент встречается чаще')
    

In [ ]:
word_1 = "мошенничество"
word_2 = "доверие"
for i in freq_nouns:
    if word_1 in i:
        quantity_1 = i[1]
    elif word_2 in i:
        quantity_2 = i[1]
        
print(f'мошенничество встречается {quantity_1} раз')
print(f'доверие встречается {quantity_2} раз')

#### 6 В поле "rating_grade" записана оценка отзыва по шкале от 1 до 5. Используйте меру  𝑡𝑓−𝑖𝑑𝑓 , для того, чтобы найти ключевые слова и биграмы для положительных отзывов (с оценкой 5) и отрицательных отзывов (с оценкой 1)

`TfidfVectorizer` делает то же, что и `CountVectorizer`, но в качестве значений – tf-idf каждого слова.

Как считается tf-idf:

TF (term frequency) – относительная частотность слова в документе:
$$ TF(t,d) = \frac{n_t}{\sum_k n_k} $$

`t` -- слово (term), `d` -- документ, $n_t$ -- количество вхождений слова, $n_k$ -- количество вхождений остальных слов

IDF (inverse document frequency) – обратная частота документов, в которых есть это слово:
$$ IDF(t, D) = \mbox{log} \frac{|D|}{|{d : t \in d}|} $$

`t` -- слово (term), `D` -- коллекция документов

Перемножаем их:
$$TFIDF_(t,d,D) = TF(t,d) \times IDF(i, D)$$

Сакральный смысл – если слово часто встречается в одном документе, но в целом по корпусу встречается в небольшом 
количестве документов, у него высокий TF-IDF.

In [127]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vec = TfidfVectorizer(ngram_range = (1, 2))
bow = vec.fit_transform(data_responses_3['text'])

### Часть 2. Тематическое моделирование
1. Постройте несколько тематических моделей коллекции документов с разным числом тем. Приведите примеры понятных (интерпретируемых) тем.
2. Найдите темы, в которых упомянуты конкретные банки (Сбербанк, ВТБ, другой банк). Можете ли вы их прокомментировать / объяснить?
Эта часть задания может быть сделана с использованием gensim.



### Часть 3. Классификация текстов
Сформулируем для простоты задачу бинарной классификации: будем классифицировать на два класса, то есть, различать резко отрицательные отзывы (с оценкой 1) и положительные отзывы (с оценкой 5).

1. Составьте обучающее и тестовое множество: выберите из всего набора данных N1 отзывов с оценкой 1 и N2 отзывов с оценкой 5 (значение N1 и N2 – на ваше усмотрение). Используйте sklearn.model_selection.train_test_split для разделения множества отобранных документов на обучающее и тестовое.
2. Используйте любой известный вам алгоритм классификации текстов для решения задачи и получите baseline. Сравните разные варианты векторизации текста: использование только униграм, пар или троек слов или с использованием символьных $n$-грам.
3. Сравните, как изменяется качество решения задачи при использовании скрытых тем в качестве признаков:
- 1-ый вариант: $tf-idf$ преобразование (sklearn.feature_extraction.text.TfidfTransformer) и сингулярное разложение (оно же – латентый семантический анализ) (sklearn.decomposition.TruncatedSVD),
- 2-ой вариант: тематические модели LDA (sklearn.decomposition.LatentDirichletAllocation).
Используйте accuracy и F-measure для оценки качества классификации.

В ноутбуке, размещенном в папке репозитория. написан примерный Pipeline для классификации текстов.

Эта часть задания может быть сделана с использованием sklearn.

## Классификация текстов

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# !!! На каждом этапе Pipeline нужно указать свои параметры
# 1-ый вариант: tf-idf + LSI
# 2-ой вариант: LDA

# clf = Pipeline([
#     ('vect', CountVectorizer(analyzer = 'char', ngram_range={4,6})),
#     ('clf', RandomForestClassifier()),
# ])



clf = Pipeline([ 
    ('vect', CountVectorizer()), 
    ('tfidf', TfidfTransformer()), 
    ('tm', TruncatedSVD()), 
    ('clf', RandomForestClassifier())
])
